In [1]:

''' 
————————————————不同训练方式的区别————————————————

训练方法    计算误差的答案   反馈的计算方式

[A]
sft        由人类生成       数学公式计算分数（交叉熵）
模型蒸馏    由模型生成       数学公式计算分数（交叉熵）

[B]
RL         无              按游戏规则打分（步骤分+结果分）
PPO        无              按游戏规则打分（（步骤分+结果分）*上一轮答题的变化幅度系数））

[C]
rlhf       无              由人类或模型打分(1-10)
DPO        无              由人类或模型打分(1 or 0)

A[通过严格输入输出训练,提高输出固定格式,负责记忆能力]
B[只针对结果和格式打分,不抠过程和细节,从而提高创造能力,泛化能力,适合开放性创造性场景]
C[由一个复杂规则打分（某种偏好,无法用具体规则描述）,捕捉到不同输出中说不出来的细微差异,
从而对齐某种偏好,提高整体的输出质量]
'''

''' 
————————————————deepseek r1 训练流程————————————————

1 [RL]
使用某一领域的专业数据（数学）去训练模型的 输出格式+答案正确性
- 选择deepseek v3 作为底座模型
- 训练输出格式[sft] -> 为了通过代码批量识别输出答案的位置。(可跳过，如果跳过，模型会在下一步中学习输出格式，会有一段时间处于摸索输出格式的无效训练时期)
- 训练结果正确性[RL] -> 提高正确率。
- 总打分 = 格式分 + 正确性分。(类似于数学考试中,步骤分+答案分)
- [RL]训练 -> deepseek r1-zero

2 [生成数据]
通过泛化能力去生成大量的 其他领域cot思维链数据(推理格式+推理答案)
(成本要远低于人类生成的数据)
- deepseek r1-zero 生成 [60万条] 并批量矫正 -> [cot推理数据]
- deepseek v3 生成 [20万条] -> [非推理数据]


3 [sft]
用上一步模型生成的数据 训练出 一个新模型
- [sft]训练zero -> deepseek r1 模型

备注:
[RL]阶段关键在于,激活模型的泛化能力,解锁 模型利用本身知识 生成 其他领域的cot格式数据 的能力。
[生成数据]阶段关键在于,如何在 格式 中 自动矫正数据,从而保证数据集质量。（人类数据肯定最好，但也更贵）
[sft],数据决定模型上限,使用人类数据上限是人类,模型蒸馏其上限是原始模型。

<<整个流程的核心在于: 如何通过 格式化输出 和 代码矫正,从而获得大量 低成本 高质量的 数据。即[RL]阶段的训练。>>

'''

''' 
————————————————[RL]阶段复现成本————————————————

基座模型:qwen2.5-0.5b  约1.1g大小
推理数据集: openai/gsm8k (全数学题)
训练配置:8000条数据 累积4次更新1次 = 约2000 step
显卡类型:h20 96g
显存占用:23.430g
训练时间:5h
开始有效学习步数:92(奖励值≠0)


基座模型:qwen2.5-1.5b 约3.4g大小
推理数据集: openai/gsm8k (全数学题)
训练配置:8000条数据 累积4次更新1次 = 约2000 step
显卡类型:h20 96g
显存占用:45.4g
训练时间:6.5h
开始有效学习步数:63(奖励值≠0)

'''

' 复现成本\n\n基座模型 qwen 0.5b\n推理数据集 openai/gsm8k (全数学题)\n8000 样本 累积 4 个样本 更新 1 次 = 2000 step\n\na100 40g\n27g显存\n4-5 小时\n'